In [1]:
import sys
import os
from os.path import abspath
import numpy as np
import pandas as pd
from utils.generate_network import generate_network
from utils.prepare_data import prepare_data
from utils.popphy_io import save_params, load_params
from utils.popphy_io import get_stat, get_stat_dict
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_curve
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from models.PopPhy import PopPhyCNN
import warnings
from datetime import datetime
import json
import warnings
warnings.filterwarnings("ignore")

import tensorflow as tf
#from models.PopPhy2 import ResNet
from models.PopPhy2 import ResNet

# Data Preparation

### Reading Configuration
Configuring which data to read in, minimun threshold needed in an OTU (individual sample must have at least set threshold relative abundance), and how many k folds for k fold cross validation.

In [2]:
dataset = 'SKG-abx'
threshold = 0
k = 5

### Reduce Features
Reduce amount of OTU features by filtering out OTUs that contain no individual sample with a relative abundance greater than the set threshold.

In [3]:
path = "../data/" + dataset
data = pd.read_csv(path + '/abundance.tsv', index_col=0, sep='\t', header=None)
to_drop = data.loc[(data < threshold).all(axis=1)]
data = data.drop(to_drop.index)

data

,1,2,3,4,5,6,7,8,9,10,...,118,119,120,121,122,123,124,125,126,127
0,,,,,,,,,,,,,,,,,,,,,
k__Bacteria|p__Bacteroidota|c__Bacteroidia|o__Bacteroidales|f__Bacteroidaceae|g__Bacteroides|s__Bacteroides_acidifaciens,35,712,107,0,616,609,1129,397,209,1490,...,2941,1171,1891,716,588,26567,18283,2126,554,20731
k__Bacteria|p__Proteobacteria|c__Gammaproteobacteria|o__Enterobacterales|f__Enterobacteriaceae|g__unclassified|s__unclassified_unclassified,0,0,9,0,0,0,0,0,0,0,...,4,2,0,0,0,9049,11919,0,0,11064
k__Bacteria|p__Firmicutes|c__Bacilli|o__Lactobacillales|f__Lactobacillaceae|g__Lactobacillus|s__Lactobacillus_murinus,294,258,318,258,403,446,914,761,6,78,...,920,721,437,509,6097,2,0,1543,1333,0
k__Bacteria|p__Firmicutes|c__Bacilli|o__Lactobacillales|f__Lactobacillaceae|g__Lactobacillus|s__Lactobacillus_johnsonii,15,135,92,470,125,54,30,26,52,151,...,1161,2464,50,1215,983,0,0,73,321,0
k__Bacteria|p__Bacteroidota|c__Bacteroidia|o__Bacteroidales|f__Rikenellaceae|g__Alistipes|s__Alistipes_unclassified,46,106,12,373,52,73,98,0,13,40,...,92,28,242,185,91,0,0,277,997,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
k__Bacteria|p__Bacteroidota|c__Bacteroidia|o__Bacteroidales|f__Bacteroidaceae|g__Bacteroides|s__Bacteroides_unclassified,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
k__Bacteria|p__Firmicutes|c__Clostridia|o__Clostridia UCG-014|f__unclassified|g__unclassified|s__unclassified_unclassified,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
k__Bacteria|p__Firmicutes|c__Clostridia|o__Clostridia vadinBB60 group|f__unclassified|g__unclassified|s__unclassified_unclassified,0,0,0,0,0,0,0,0,0,0,...,6,0,0,0,0,0,0,0,0,0


### Create 2d Matrix Representing OTU Data
Dai et al. PopPhy-CNN's (2019) algorithm creates Phylogenetic tree from OTUs and populates tree based on OTU abundances. This tree graph structure is then converted to a 2d Matrix by taking each parent node in the tree graph and pushing them all to the left and childrens' nodes in the same order from left to right the parents were ordered.

In [4]:
my_maps, raw_x, tree_x, raw_features, tree_features, labels, label_set, g, feature_df = prepare_data(path, data)

pd.DataFrame(my_maps[0])

(692, 127)
There are 692 raw features...
Building tree structure...
Found tree file...
Populating trees...
There are 337 tree features...


,0,1,2,3,4,5,6,7,8,9,...,580,581,582,583,584,585,586,587,588,589
0,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.014875,0.357585,0.046097,0.001915,0.078645,0.498527,0.002356,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.014875,0.100589,0.031811,0.000000,0.225184,0.046097,0.000000,0.001915,0.078645,0.465685,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.014875,0.006186,0.094404,0.031811,0.000000,0.000000,0.000000,0.000000,0.000000,0.225184,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.014875,0.006186,0.094404,0.000295,0.031517,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.014875,0.006186,0.094404,0.000295,0.031517,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.014875,0.000000,0.006186,0.000000,0.000000,0.000000,0.000000,0.094404,0.000295,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.001473,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Creating training and test sets
Splitting data into k training and k test sets

In [5]:
#one hot encoding
input = my_maps
target = tf.keras.utils.to_categorical(labels, 2, dtype='int64')
    

#shuffle dataset
seed = np.random.randint(100)
# np.random.seed(seed)
# np.random.shuffle(input)
np.random.seed(seed)
np.random.shuffle(target)

np.random.seed(seed)
np.random.shuffle(my_maps)
np.random.seed(seed)
np.random.shuffle(raw_x)
np.random.seed(seed)
np.random.shuffle(tree_x)
np.random.seed(seed)
np.random.shuffle(labels)


#create k training and k test sets
groups_input = []
groups_target = []
k_size = len(input)//k
start, end = 0, k_size
for i in range(k):
    if i == k-1:
        group_input = input[start:]
        group_target = target[start:]
    else:
        group_input = input[start:end]
        group_target = target[start:end]
    start += k_size
    end += k_size
    groups_input.append(group_input)
    groups_target.append(group_target)

x_train = []
y_train = []
x_test = []
y_test = []
for i in range(k-1, -1, -1):
    x_train.append(np.concatenate((groups_input[i-1], groups_input[i-2], groups_input[i-3], groups_input[i-4])))
    y_train.append(np.concatenate((groups_target[i-1], groups_target[i-2], groups_target[i-3], groups_target[i-4])))

    x_test.append(groups_input[i])
    y_test.append(groups_target[i])

# Model

### Training model
Data is log transformed and then a MinMax transformation. Uses CNN that employs skipped residual identity blocks borrowed from the classic ResNet model then a FC Neural Network to make phenotype prediction. Model dimensions printed below.

In [6]:
data_lst = []

# for i in range(k):
#     x_train1 = x_train[i]
#     y_train1 = y_train[i]
#     x_test1 = x_test[i]
#     y_test1 = y_test[i]

#     model = ResNet(height = x_train1.shape[1], width = x_train1.shape[2], channels = 1, classes = 2)
#     model.init_model()

#     model.train(x_train1, y_train1, x_test1, y_test1, dataset, use_weights = False)
#     y_pred = model.predict(x_test1)
#     auc_roc, auc_pr, f1, mcc = model.evaluate(y_test1, y_pred)
#     data_lst.append([auc_roc, auc_pr, f1, mcc])
    
#     #model.model.save_weights(path + "/model_weights.h5")

#     print(y_test1)
#     print(y_pred)
    
# print(model.model.summary())



n_values = np.max(labels) + 1
labels_oh = np.eye(n_values)[labels]
tree_row = my_maps.shape[1]
tree_col = my_maps.shape[2]

skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=seed)
fold = 0
for train_index, test_index in skf.split(my_maps, labels):
    train_x, test_x = my_maps[train_index,:,:], my_maps[test_index,:,:]
    train_y, test_y = labels_oh[train_index,:], labels_oh[test_index,:]
        
    train_x = np.log(train_x + 1)
    test_x = np.log(test_x + 1)
        
    c_prob = [0] * len(np.unique(labels))
    train_weights = []

    for l in np.unique(labels):
        a = float(len(labels))
        b = 2.0 * float((np.sum(labels==l)))
        c_prob[int(l)] = a/b

    c_prob = np.array(c_prob).reshape(-1)

    for l in np.argmax(train_y, 1):
        train_weights.append(c_prob[int(l)])
    train_weights = np.array(train_weights)
        
    scaler = MinMaxScaler().fit(train_x.reshape(-1, tree_row * tree_col))
    train_x = np.clip(scaler.transform(train_x.reshape(-1, tree_row * tree_col)), 0, 1).reshape(-1, tree_row, tree_col)
    test_x = np.clip(scaler.transform(test_x.reshape(-1, tree_row * tree_col)), 0, 1).reshape(-1, tree_row, tree_col)

    train = [train_x, train_y]
    test = [test_x, test_y]

    x_train1 = train_x
    y_train1 = train_y
    x_test1 = test_x
    y_test1 = test_y
        
#         y_train1 = train_y
#         y_test1 = test_y
        
#         x_train1 = np.zeros(train_x.shape)
#         x_train1[train_x != 0] = 1
        
#         x_test1 = np.zeros(test_x.shape)
#         x_test1[test_x != 0] = 1
        
        # for i in range(len(train_x)):
        #     for j in range(len(test_x)):
        #         if np.array_equal(train_x[i], test_x[j]):
        #             print('train')
        #             print(train_x[i])
        #             print('test')
        #             print(test_x[j])
        
        
    model = ResNet(height = train_x.shape[1], width = train_x.shape[2], channels = 1, classes = 2)
    model.init_model()
    model.train(train_x, train_y, test_x, y_test1, dataset, use_weights = False)
    y_pred = model.predict(test_x)
    auc_roc, auc_pr, f1, mcc = model.evaluate(test_y, y_pred)
    data_lst.append([auc_roc, auc_pr, f1, mcc])
    #model.model.save_weights(path + "/model_weights.h5")
    print(test_y)
    print(y_pred)
    print(model.model.summary())
    
    fold += 1
#run += 1
        
        
        
        

Metal device set to: Apple M1 Pro
Epoch 1/100
5/5 [==============================] - 2s 225ms/step - loss: 3.5876 - accuracy: 0.6040 - val_loss: 2.4492 - val_accuracy: 0.6538
Epoch 2/100
5/5 [==============================] - 0s 104ms/step - loss: 2.1127 - accuracy: 0.8317 - val_loss: 2.3216 - val_accuracy: 0.6538
Epoch 3/100
5/5 [==============================] - 0s 101ms/step - loss: 1.8159 - accuracy: 0.9109 - val_loss: 2.1808 - val_accuracy: 0.6538
Epoch 4/100
5/5 [==============================] - 0s 89ms/step - loss: 1.6126 - accuracy: 0.9406 - val_loss: 2.0335 - val_accuracy: 0.6538
Epoch 5/100
5/5 [==============================] - 0s 105ms/step - loss: 1.3874 - accuracy: 0.9802 - val_loss: 1.8954 - val_accuracy: 0.7308
Epoch 6/100
5/5 [==============================] - 0s 103ms/step - loss: 1.2257 - accuracy: 0.9802 - val_loss: 1.7771 - val_accuracy: 0.6923
Epoch 7/100
5/5 [==============================] - 0s 92ms/step - loss: 1.0894 - accuracy: 1.0000 - val_loss: 1.6742 - va

# Displaying Accuracy Metrics and Saving Metrics

Option to save results of all k folds and weights of last model into same directy as data.

In [7]:
col = [str(i) for i in range(1,k+1)]    
results_df = pd.DataFrame(data_lst, columns = ['auc(roc)', 'auc(pr)', 'f1', 'mcc'])
results_df = results_df.transpose()
results_df.columns = col

#results_df.to_csv(path + "/results.csv")
results_df

,1,2,3,4,5
auc(roc),0.915033,0.888889,0.895833,0.911765,1.000000
auc(pr),0.855934,0.888860,0.896192,0.907709,1.000000
f1,0.849883,0.769231,0.842667,0.821754,0.845333
mcc,0.727607,0.490196,0.678234,0.636209,0.714006


# Saving Model Weights

Option to save model weights of last model in k fold into same directy as data.

In [8]:
#model.model.save_weights(path + "/model_weights.h5")

## 